# IMPORT LIBRARY 
- mount Google Drive 
- LightGBM 
- selenium (chrome.exe) 

In [5]:
# 테스트 
from lightgbm import LGBMClassifier, plot_importance


In [199]:
# import libarary 
import os 
import shutil 
import random 
import numpy as np 
import time 
from collections import Counter 
from sklearn.metrics import mean_squared_error 

# data pre-processing 
import pandas as pd 
pd.set_option('max_columns', 130, 'max_rows', 30)

#visualization 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 
plt.rc('font', family = 'Malgun Gothic')
plt.rc('axes', unicode_minus = False)

#ignore warning 
import warnings
warnings.filterwarnings('ignore')

import folium #지도 관련 시각화 
from folium.plugins import MarkerCluster #지도 관련 시각화 
import geopy.distance #거리 계산해주는 패키지 사용 

# save 
from sklearn.externals import joblib 
import pickle 

# selenium 
# from selenium.webdriver import Chrome 

# UsageError: Line magic function `%matplotlib.inline` not found. 
import IPython
print(IPython.sys_info())

import geopy.distance 

from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestRegressor 

import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 
from sklearn.externals import joblib 
import pickle 

import random 
random.seed(42)

import lightgbm as lgb 
from keras import metrics 
from sklearn.metrics import r2_score 

{'commit_hash': '8bda98619',
 'commit_source': 'installation',
 'default_encoding': 'cp949',
 'ipython_path': 'D:\\CONDA\\lib\\site-packages\\IPython',
 'ipython_version': '7.12.0',
 'os_name': 'nt',
 'platform': 'Windows-10-10.0.18362-SP0',
 'sys_executable': 'D:\\CONDA\\python.exe',
 'sys_platform': 'win32',
 'sys_version': '3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit '
                '(AMD64)]'}


In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
bts = pd.read_csv("bus_bts.csv")

train.shape, test.shape, bts.shape

((415423, 21), (228170, 20), (2409414, 13))

Read extenral crawled data 

In [6]:
# 날씨 데이터 
weather_data = pd.read_csv('weather.csv')

weather_data.shape

(46, 4)

In [7]:
# 제주 공항 도착 고객 데이터 
airport_data = pd.read_csv('df_airport.csv')

airport_data.shape

(49, 3)

In [14]:
rain3 = pd.read_csv('rain3.csv', encoding = ' utf-8')

# 데이터 전처리 

In [19]:
# 요일 
train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday 

test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday 

train.shape, test.shape

((415423, 22), (228170, 21))

In [21]:
# route_station = bus_route_id + station_code 
train['bus_route_id'] = train['bus_route_id'].astype(str)
train['station_code'] = train['station_code'].astype(str)
train['route_station'] = train['bus_route_id'] + ',' + train['station_code']

test['bus_route_id'] = test['bus_route_id'].astype(str)
test['station_code'] = test['station_code'].astype(str)
test['route_station'] = test['bus_route_id'] + ',' + test['station_code']

train.shape, test.shape

((415423, 23), (228170, 22))

In [22]:
# bus_route_id_weekday = bus_route_id + weekday 
train['bus_route_id_weekday'] = train['bus_route_id'].astype(str) + ',' + train['weekday'].astype(str)
test['bus_route_id_weekday'] = test['bus_route_id'].astype(str) + ',' + test['weekday'].astype(str)

train.shape, test.shape

((415423, 24), (228170, 23))

In [23]:
# station_code_weekday = station_code + weekday 

train['station_code_weekday'] = train['station_code'].astype(str) + ',' + train['weekday'].astype(str)
test['station_code_weekday'] = test['station_code'].astype(str) + ',' + test['weekday'].astype(str)

train.shape, test.shape

((415423, 25), (228170, 24))

In [24]:
# route_station_weekday = route_station + weekday 
train['route_station_weekday'] = train['route_station'].astype(str) + ',' + train['weekday'].astype(str)
test['route_station_weekday'] = test['route_station'].astype(str) + ',' + test['weekday'].astype(str)

train.shape, test.shape 

((415423, 26), (228170, 25))

In [25]:
# on_time
# bts.csv 데이터에서 geton_time 열에서 시간대만 추출하여 on_time 컬럼을 만듦 

bts['on_time'] = bts['geton_time'].apply(lambda x: x[:2]) # 첫 두번째 string slicing 

bts.iloc[bts.query('on_time == "06"').index, 13] = '6~7_ride'
bts.iloc[bts.query('on_time == "07"').index, 13] = '7~8_ride'
bts.iloc[bts.query('on_time == "08"').index, 13] = '8~9_ride'
bts.iloc[bts.query('on_time == "09"').index, 13] = '9~10_ride'
bts.iloc[bts.query('on_time == "10"').index, 13] = '10~11_ride'
bts.iloc[bts.query('on_time == "11"').index, 13] = '11~12_ride'

bts.shape 

(2409414, 14)

In [26]:
# 승 하차 시간대 통합 변수 (t~t+2)
# t~t+1, t+1~t+2 시간대 승하차인원을 합하여 t~t+2 시간대 승하차인원 변수를 만듬
train['68a']=train['6~7_ride']+train['7~8_ride'] 
train['810a']=train['8~9_ride']+train['9~10_ride']
train['1012a']=train['10~11_ride']+train['11~12_ride']

train['68b']=train['6~7_takeoff']+train['7~8_takeoff'] 
train['810b']=train['8~9_takeoff']+train['9~10_takeoff']
train['1012b']=train['10~11_takeoff']+train['11~12_takeoff']

test['68a']=test['6~7_ride']+test['7~8_ride']
test['810a']=test['8~9_ride']+test['9~10_ride']
test['1012a']=test['10~11_ride']+test['11~12_ride']

test['68b']=test['6~7_takeoff']+test['7~8_takeoff']
test['810b']=test['8~9_takeoff']+test['9~10_takeoff']
test['1012b']=test['10~11_takeoff']+test['11~12_takeoff']

train.shape, test.shape

((415423, 32), (228170, 31))

Make features by using target variable
우리가 최종적으로 예측해야할 것은 각 일자별(date), 버스 노선(bus_route_id) 상의 정류장(station_name)의 퇴근시간 하차인원(18~20_ride)이다.

- bus_route_id, station_name, weekday의 각 조합별 퇴근시간 하차인원(18~20_ride)의 여러 통계량을 구한 후 이를 train set, test set에 모두 적용한다."

- target 변수를 train, test set에 적용할 수 있는 이유는 우리가 예측해야할 id는 date, bus_rout_id, station_name으로 구성되어있기 때문이다. 즉, 각각의 노선, 정류장별로 공통적인 패턴이 존재할 수 있다.

- 이 과정에서 NA 값이 생기는 이유는 train set에 없는 bus_route_id, station_name이 존재하기 때문이다.

In [27]:
def id_statistic(ID, col1, col2): 

    # mean, sum 
    rs_mean = train.groupby([ID])['18~20_ride'].agg([(col1, 'mean')]).reset_index()
    rs_sum = train.groupby([ID])['18~20_ride'].agg([(col2, 'sum')]).reset_index()
    rs_mean_sum = pd.merge(rs_mean, rs_sum, on = ID)

    # merge 
    tr = pd.merge(train, rs_mean_sum, how = 'left', on = ID)
    te = pd.merge(test, rs_mean_sum, how = 'left', on = ID)

    # na -> mean 
    # 결측값을 mean으로 대체해주며 처리해주었다 
    te[col1] = te[col1].fillna(rs_mean.mean())
    te[col2] = te[col2].fillna(rs_sum.mean())

    return tr, te 

In [28]:
train, test = id_statistic('route_station', '1820_rs_mean', '1820_rs_sum')

train.shape, test.shape

((415423, 34), (228170, 33))

In [29]:
train, test = id_statistic('bus_route_id', '1820_r_mean', '1820_r_sum')

train.shape, test.shape

((415423, 36), (228170, 35))

In [30]:
train, test = id_statistic('station_code', '1820_s_mean', '1820_s_sum')

train.shape, test.shape

((415423, 38), (228170, 37))

In [31]:
train, test = id_statistic('weekday','1820_w_mean','1820_w_sum')

train.shape, test.shape

((415423, 40), (228170, 39))

In [32]:
def mean_statistics() :

    f = train.groupby(['bus_route_id_weekday'])['18~20_ride'].agg([('mean_bus_weekday_ride','mean')]).reset_index()
    tr = pd.merge(train, f, how='left', on='bus_route_id_weekday')
    te = pd.merge(test, f, how='left', on='bus_route_id_weekday').fillna(f['mean_bus_weekday_ride'].mean())
    
    f = train.groupby(['station_code_weekday'])['18~20_ride'].agg([('mean_station_weekday_ride','mean')]).reset_index()
    tr = pd.merge(tr, f, how='left', on='station_code_weekday')
    te = pd.merge(te, f, how='left', on='station_code_weekday').fillna(f['mean_station_weekday_ride'].mean())
    
    f = train.groupby(['route_station_weekday'])['18~20_ride'].agg([('mean_route_station_weekday_ride','mean')]).reset_index()
    tr = pd.merge(tr, f, how='left', on='route_station_weekday')
    te = pd.merge(te, f, how='left', on='route_station_weekday').fillna(f['mean_route_station_weekday_ride'].mean())
    
    return tr, te

In [33]:
train, test = mean_statistics()

train.shape, test.shape

((415423, 43), (228170, 42))

In [34]:
# congestion 
# bus_route_id를 기준으로 18~20의 혼잡도를 계산한다 

def congestion() :
    df = train.groupby(['bus_route_id'])['18~20_ride'].agg([('passenger', 'sum')]) # bus_route_id 를 18~20에 타는 승객들을 합산해줌으로써 얼마나 붐비는지 알 수 있음 
    df = df.sort_values(by='passenger', ascending=False).reset_index()
    
    def f(x):
        if x > 10000:
            return 7

        elif x > 5000:
            return 6

        elif x > 2000:
            return 5

        elif x > 700:
            return 4

        elif x > 200:
            return 3

        elif x > 50:
            return 2

        else:
            return 1
    
    df['congestion']=df['passenger'].apply(f)
    df = df[['bus_route_id','congestion']]
    
    tr = pd.merge(train, df, how='left', on='bus_route_id')
    te = pd.merge(test, df, how='left', on='bus_route_id')
    
    # 결측치는 데이터 프레임 df의 'congestion'의 중간값인 '4'으로 대체
    te = te.fillna(4)
    
    return tr, te

In [35]:
train, test = congestion()

train.shape, test.shape

((415423, 44), (228170, 43))

In [36]:
# location = latitude + longitude


train['location'] = train['latitude'].astype(str) + ',' + train['longitude'].astype(str)
test['location'] = test['latitude'].astype(str) + ',' + test['longitude'].astype(str)

train.shape, test.shape

((415423, 45), (228170, 44))

In [37]:
# merge key 
# make cue column
train['cue']=0
test['cue']=1

train.shape, test.shape


((415423, 46), (228170, 45))

In [38]:
def morning():  
    # merge
    data = pd.concat([train, test])
    
    a = data.groupby(['route_station'])['1012a'].agg({'sum', 'mean'}).reset_index()
    a.columns = ['route_station', '1012a_sum','1012a_mean']

    b = data.groupby(['route_station'])['1012b'].agg({'sum', 'mean'}).reset_index()
    b.columns = ['route_station', '1012b_sum','1012b_mean']
    b = b[['1012b_sum','1012b_mean']]

    c = data.groupby(['route_station'])['10~11_ride'].agg({'sum', 'mean'}).reset_index()
    c.columns = ['route_station', '10~11_ride_sum','10~11_ride_mean']
    c = c[['10~11_ride_sum','10~11_ride_mean']]

    d = data.groupby(['route_station'])['10~11_takeoff'].agg({'sum', 'mean'}).reset_index()
    d.columns = ['route_station', '10~11_takeoff_sum','10~11_takeoff_mean']
    d = d[['10~11_takeoff_sum','10~11_takeoff_mean']]

    e = data.groupby(['route_station'])['11~12_ride'].agg({'sum', 'mean'}).reset_index()
    e.columns = ['route_station', '11~12_ride_sum','11~12_ride_mean']
    e = e[['11~12_ride_sum','11~12_ride_mean']]

    f = data.groupby(['route_station'])['11~12_takeoff'].agg({'sum', 'mean'}).reset_index()
    f.columns = ['route_station', '11~12_takeoff_sum','11~12_takeoff_mean']
    f = f[['11~12_takeoff_sum','11~12_takeoff_mean']]

    g = data.groupby(['route_station'])['1820_r_mean'].agg({'sum', 'mean'}).reset_index()
    g.columns = ['route_station', '1820_r_mean_sum','1820_r_mean_mean']
    g = g[['1820_r_mean_sum','1820_r_mean_mean']]

    h = data.groupby(['route_station'])['1820_r_sum'].agg({'sum', 'mean'}).reset_index()
    h.columns = ['route_station', '1820_r_sum_sum','1820_r_sum_mean']
    h = h[['1820_r_sum_sum','1820_r_sum_mean']]

    i = data.groupby(['route_station'])['1820_rs_mean'].agg({'sum', 'mean'}).reset_index()
    i.columns = ['route_station', '1820_rs_mean_sum','1820_rs_mean_mean']
    i = i[['1820_rs_mean_sum','1820_rs_mean_mean']]

    j = data.groupby(['route_station'])['1820_rs_sum'].agg({'sum', 'mean'}).reset_index()
    j.columns = ['route_station', '1820_rs_sum_sum','1820_rs_sum_mean']
    j = j[['1820_rs_sum_sum','1820_rs_sum_mean']]

    k = data.groupby(['route_station'])['1820_s_mean'].agg({'sum', 'mean'}).reset_index()
    k.columns = ['route_station', '1820_s_mean_sum','1820_s_mean_mean']
    k = k[['1820_s_mean_sum','1820_s_mean_mean']]

    l = data.groupby(['route_station'])['1820_s_sum'].agg({'sum', 'mean'}).reset_index()
    l.columns = ['route_station', '1820_s_sum_sum','1820_s_sum_mean']
    l = l[['1820_s_sum_sum','1820_s_sum_mean']]

    m = data.groupby(['route_station'])['1820_w_mean'].agg({'sum', 'mean'}).reset_index()
    m.columns = ['route_station', '1820_w_mean_sum','1820_w_mean_mean']
    m = m[['1820_w_mean_sum','1820_w_mean_mean']]

    n = data.groupby(['route_station'])['1820_w_sum'].agg({'sum', 'mean'}).reset_index()
    n.columns = ['route_station', '1820_w_sum_sum','1820_w_sum_mean']
    n = n[['1820_w_sum_sum','1820_w_sum_mean']]

    o = data.groupby(['route_station'])['68a'].agg({'sum', 'mean'}).reset_index()
    o.columns = ['route_station', '68a_sum','68a_mean']
    o = o[['68a_sum','68a_mean']]

    p = data.groupby(['route_station'])['68b'].agg({'sum', 'mean'}).reset_index()
    p.columns = ['route_station', '68b_sum','68b_mean']
    p = p[['68b_sum','68b_mean']]

    q = data.groupby(['route_station'])['6~7_ride'].agg({'sum', 'mean'}).reset_index()
    q.columns = ['route_station', '6~7_ride_sum','6~7_ride_mean']
    q = q[['6~7_ride_sum','6~7_ride_mean']]

    r = data.groupby(['route_station'])['6~7_takeoff'].agg({'sum', 'mean'}).reset_index()
    r.columns = ['route_station', '6~7_takeoff_sum','6~7_takeoff_mean']
    r = r[['6~7_takeoff_sum','6~7_takeoff_mean']]

    s = data.groupby(['route_station'])['7~8_ride'].agg({'sum', 'mean'}).reset_index()
    s.columns = ['route_station', '7~8_ride_sum','7~8_ride_mean']
    s = s[['7~8_ride_sum','7~8_ride_mean']]

    t = data.groupby(['route_station'])['7~8_takeoff'].agg({'sum', 'mean'}).reset_index()
    t.columns = ['route_station', '7~8_takeoff_sum','7~8_takeoff_mean']
    t = t[['7~8_takeoff_sum','7~8_takeoff_mean']]

    u = data.groupby(['route_station'])['810a'].agg({'sum', 'mean'}).reset_index()
    u.columns = ['route_station', '810a_sum','810a_mean']
    u = u[['810a_sum','810a_mean']]

    v = data.groupby(['route_station'])['810b'].agg({'sum', 'mean'}).reset_index()
    v.columns = ['route_station', '810b_sum','810b_mean']
    v = v[['810b_sum','810b_mean']]

    w = data.groupby(['route_station'])['8~9_ride'].agg({'sum', 'mean'}).reset_index()
    w.columns = ['route_station', '8~9_ride_sum','8~9_ride_mean']
    w = w[['8~9_ride_sum','8~9_ride_mean']]

    x = data.groupby(['route_station'])['8~9_takeoff'].agg({'sum', 'mean'}).reset_index()
    x.columns = ['route_station', '8~9_takeoff_sum','8~9_takeoff_mean']
    x = x[['8~9_takeoff_sum','8~9_takeoff_mean']]

    y = data.groupby(['route_station'])['9~10_ride'].agg({'sum', 'mean'}).reset_index()
    y.columns = ['route_station', '9~10_ride_sum','9~10_ride_mean']
    y = y[['9~10_ride_sum','9~10_ride_mean']]

    z = data.groupby(['route_station'])['9~10_takeoff'].agg({'sum', 'mean'}).reset_index()
    z.columns = ['route_station', '9~10_takeoff_sum','9~10_takeoff_mean']
    z = z[['9~10_takeoff_sum','9~10_takeoff_mean']]
    
    df = pd.concat([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z],axis=1)
    df = pd.merge(data, df, how='left', on='route_station')
    
    return df



In [39]:
data = morning()

data.shape

(643593, 98)

In [40]:
# 배차 간격 
train['bus_route_id'] = train['bus_route_id'].astype(np.int64)
test['bus_route_id'] = test['bus_route_id'].astype(np.int64)

In [41]:
bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

f = bts.groupby(['geton_date','geton_time2','geton_station_code','bus_route_id'])['user_count'].\
agg([('탑승객_수','sum')]).reset_index().\
sort_values(by=['geton_date','geton_station_code','bus_route_id','geton_time2'], ascending=True).reset_index()

f['index'] = list(range(0,len(f)))

In [42]:
# time = []

# for i in range(0,len(f)-1):

#     if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) &\
#         (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) &\
#         (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):

#         time.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)

#     else:
#         time.append(0)

# time.insert(0, '0')


In [43]:
# def get_sec(time_str):

#     h, m, s = time_str.split(':')

#     return int(h) * 3600 + int(m) * 60 + int(s)

In [44]:
# def bus_interval() :

#     f['time'] = time
#     f['time2'] = f['time'].astype(str).str[7:]


#     interval = f.copy()
#     interval['time2'] = interval['time2'].astype(str).replace('','00:00:00')
#     interval['bus_route_id'] = interval['bus_route_id'].astype(object)

#     time4 = []

#     for i in interval['time2'] :

#         time4.append(get_sec(i))

#     interval['time4'] = time4
#     interval['time4'] = (interval['time4'] / 60).astype(int)

#     interval = interval[interval['time4'] > 3] # 간격이 3분보다 작은 것 제외 
#     interval = interval[interval['time4'] < 180] # 간격이 3시간보다 큰 것 제외

#     interval = interval.groupby('bus_route_id')['time4'].agg([('bus_interval', 'mean')]).reset_index()
#     interval['bus_interval'] = interval['bus_interval'].astype(int)

#     # 나중에 시간을 절약하기 위해 csv 파일로 저장
#     interval.to_csv('bus_interval_final.csv', index = False)

#     print('success.. !')

In [45]:

# bus_interval()


success.. !


In [46]:
# 이 과정은 상당히 오래걸리는 관계로 따로 데이터셋을 만들어서 구성해놓았습니다.
bus_interval = pd.read_csv("bus_interval_final.csv")
data['bus_route_id'] = data['bus_route_id'].astype(np.int64)
data = pd.merge(data, bus_interval, how = 'left', on = 'bus_route_id')

In [47]:
# 'bus_interval'컬럼에서 na값을 가지는 행들을 분석해본 결과, 대부분 18~20시의 탑승 인원이 거의 없는 bus_route_id 와 station_code 였다. 따라서 탑승 인원이 별로 없을 것이라고 예상되는 버스는 배차 간격이 길 것이라고 판단하여 na값을 '9999' 로 채워주었다.
data['bus_interval'] = data['bus_interval'].fillna(9999)

In [48]:
# Label encoding feature
# bus_route_id, station_name, route_station_weekday(bus_route_id + weekday의 조합), route_station(bus_route_id + station_code 의 조합) 총 4개를 라벨인코딩 해줌

labelencoder = LabelEncoder()
df_encode = data[['bus_route_id','station_code', 'route_station_weekday', 'route_station']]
df_encoded = df_encode.apply(labelencoder.fit_transform)

data['bus_route_id2']=df_encoded['bus_route_id']
data['station_code2']=df_encoded['station_code']
data['route_station_weekday2']=df_encoded['route_station_weekday']
data['route_station2']=df_encoded['route_station']

data.shape

(643593, 103)

In [49]:
# weather 

def weather() :

    weather_data = pd.read_csv('weather.csv', encoding = 'utf-8')
    weather_data['id'] = range(0,46)

    a = pd.DataFrame(data.date.unique(), columns=['date']) ; a['id'] = range(0,46)
    weather_data = pd.merge(a, weather_data)
    weather_data = weather_data[['date','현재일기_10','체감온도_10','일강수_10']]
    weather_data = weather_data.replace(' ', 0)
    df = pd.merge(data, weather_data, on='date')
    # label_encoder
    labelencoder = LabelEncoder()
    df_encode = df[['현재일기_10']]
    df_encoded = df_encode.apply(labelencoder.fit_transform)
    df['현재일기_10']=df_encoded['현재일기_10']
    # object->float 변수변환
    df['현재일기_10'] = df['현재일기_10'].astype(float)
    df['체감온도_10'] = df['체감온도_10'].astype(float)
    df['일강수_10'] = df['일강수_10'].astype(float)
    return df

In [50]:
weather_data = pd.read_csv('weather.csv')
weather_data.shape

(46, 4)

In [51]:
data = weather()
data.shape

(643593, 106)

In [52]:
# weekday(data)
data['date'] = pd.to_datetime(data['date'])
data['weekday'] = data['date'].dt.weekday
data = pd.get_dummies(data,columns=['weekday'])
data['weekday'] = data['date'].dt.weekday
data.shape

(643593, 113)

In [53]:
# in-out

data['in_out'].value_counts()
data['in_out'] = data['in_out'].map({'시내':0,'시외':1})

data.shape

(643593, 113)

In [54]:
#  
# 좌표데이터를 이용 변수
# 좌표데이터를 이용한 변수를 만들 때는 많은 시간이 소요됩니다.


coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
data['dis_jejusi'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusi).km for i in range(len(data))]

coords_jejusicheong1 = (33.49892, 126.53035) #제주시청(광양방면)의 위도 경도
coords_jejuairport = (33.50661, 126.49345) #제주국제공항(구제주방면)의 위도 경도
coords_hallahosp = (33.48963, 126.486) #한라병원의 위도 경도
coords_rotary = (33.49143, 126.49678) # 제주도청신제주로터리의 위도 경도
coords_jejucenterhigh = (33.48902, 126.5392) #제주중앙여자고등학교의 위도 경도
coords_jejumarket = (33.51315, 126.52706) #동문시장의 위도 경도
coords_jejusclass = (33.47626, 126.48141) #제주고등학교/중흥S클래스의 위도 경도
coords_centerroad = (33.51073, 126.5239) #중앙로(국민은행)의 위도 경도
coords_fiveway = (33.48667, 126.48092) # 노형오거리의 위도 경도
coords_law = (33.49363, 126.53476) # 제주지방법원(광양방면)의 위도 경도

data['dis_jejusicheong1'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusicheong1).km for i in range(len(data))]
data['dis_jejuairport'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejuairport).km for i in range(len(data))]
data['dis_hallahosp'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_hallahosp).km for i in range(len(data))]
data['dis_rotary'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_rotary).km for i in range(len(data))]
data['dis_jejucenterhigh'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejucenterhigh).km for i in range(len(data))]
data['dis_jejumarket'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejumarket).km for i in range(len(data))]
data['dis_jejusclass'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusclass).km for i in range(len(data))]
data['dis_centerroad'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_centerroad).km for i in range(len(data))]
data['dis_fiveway'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_fiveway).km for i in range(len(data))]
data['dis_law'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_law).km for i in range(len(data))]

data.shape

(643593, 124)

In [55]:
# 출근시간 총 승객 수 
data['ride_sum'] = data['6~7_ride'] + data['7~8_ride'] + data['8~9_ride'] + data['9~10_ride'] + data['10~11_ride'] + data['11~12_ride'] 
data['takeoff_sum'] = data['6~7_takeoff'] + data['7~8_takeoff'] + data['8~9_takeoff'] + data['9~10_takeoff'] + data['10~11_takeoff'] + data['11~12_takeoff'] 

data.shape

(643593, 126)

In [56]:
# 날짜 및 시간대 별 총 승객 수 
f = data.groupby('date')['6~7_ride'].agg([('6~7_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['7~8_ride'].agg([('7~8_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['8~9_ride'].agg([('8~9_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['9~10_ride'].agg([('9~10_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['10~11_ride'].agg([('10~11_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

data.shape

(643593, 131)

In [57]:
# 주말, 주중 
def h(x):
    if x ==5:
        return 1
    elif x==6:
        return 1
    else:
        return 0

In [58]:
data['weekend'] = data['weekday'].apply(h)

data.shape

(643593, 132)

In [59]:
# 연휴 
def g(x):
    if x in ['2019-09-12','2019-09-13','2019-09-14','2019-10-03','2019-10-09']:
        return 1
    else:
        return 0

In [60]:
data['holiday'] = data['date'].apply(g) 

data.shape

(643593, 133)

In [61]:
# 요일 별 평균 승객 수 

def week_mean() :

    df = data.reset_index(drop=True)
    df.groupby('weekday')['18~20_ride'].mean()
    df['weekdaymean']= 1

    index0 = df.query('weekday==0').index
    index1 = df.query('weekday==1').index
    index2 = df.query('weekday==2').index
    index3 = df.query('weekday==3').index
    index4 = df.query('weekday==4').index
    index5 = df.query('weekday==5').index
    index6 = df.query('weekday==6').index

    df.iloc[index0,-1] = 1.343710
    df.iloc[index1,-1] = 1.375319
    df.iloc[index2,-1] = 1.430856
    df.iloc[index3,-1] = 1.256710
    df.iloc[index4,-1] = 1.067439
    df.iloc[index5,-1] = 1.062123
    df.iloc[index6,-1] = 1.034282

    return df

In [62]:
data = week_mean()

data.shape

(643593, 134)

In [63]:
# 시내 및 시외버스 별 평균 탑승 승객 
data['in_out_mean'] = 1
inindex = data.query('in_out == "시내"').index
outindex = data.query('in_out == "시외"').index

data.iloc[inindex,-1] = 1.228499
data.iloc[outindex,-1] = 2.044345
data['congestion'] = data['congestion'].astype('int64')

data.shape

(643593, 135)

In [64]:
# 카테고리별 승객 수 
def category_people() :
    bts['bus_route_id'] = bts['bus_route_id'].astype(str)

    f = bts.groupby(['bus_route_id','user_category'])['user_count'].agg([('승객수', 'sum')]).reset_index()

    g = pd.pivot_table(f, values='승객수', index='bus_route_id', columns='user_category',fill_value=0).reset_index()
    g.columns = ['bus_route_id', 'adult','kids','teen','elder','jang','jang2','ugong','ugong2']
    g = g[['bus_route_id', 'adult','kids','teen','elder']]

    # merge
    df = pd.merge(data, g, how='left', on='bus_route_id')

    # na preprocessing -> mean value
    df['adult'] = df['adult'].fillna(2363.077778)
    df['kids'] = df['kids'].fillna(60.426984)
    df['teen'] = df['teen'].fillna(448.277778)
    df['elder'] = df['elder'].fillna(751.309524)
                 
    return df

In [65]:
data['bus_route_id'] = data['bus_route_id'].astype(str)

In [66]:
data = category_people()

data.shape

(643593, 139)

In [67]:
# Category별 승객의 비율

def category_people_ratio() :
    
    a = bts.groupby('bus_route_id')['user_count'].agg([('전체', 'sum')]).reset_index()
    b = bts.groupby(['bus_route_id','user_category'])['user_count'].agg([('승객수', 'sum')]).reset_index()

    c = pd.merge(b, a, on='bus_route_id')
    c['비율'] = c['승객수']/c['전체']
    c = pd.pivot_table(c, values='비율', index='bus_route_id', columns='user_category',fill_value=0).reset_index()
    c.columns = ['bus_route_id', 'adult_prop','kids_prop','teen_prop','elder_prop','jang_prop','jang2_prop','ugong_prop','ugong2_prop']
    f = c[['bus_route_id', 'adult_prop','kids_prop','teen_prop','elder_prop']]

    df = pd.merge(data, f, how='left', on='bus_route_id')

    # na preprocessing -> mean value
    df['adult_prop'] = df['adult_prop'].fillna(0.549702)
    df['kids_prop'] = df['kids_prop'].fillna(60.426984)
    df['teen_prop'] = df['teen_prop'].fillna(0.019902)
    df['elder_prop'] = df['elder_prop'].fillna(0.235848)

    return df

In [68]:
data = category_people_ratio()

data.shape

(643593, 143)

In [84]:
for col in data.columns:
    print(col)

id
date
bus_route_id
in_out
station_code
station_name
latitude
longitude
6~7_ride
7~8_ride
8~9_ride
9~10_ride
10~11_ride
11~12_ride
6~7_takeoff
7~8_takeoff
8~9_takeoff
9~10_takeoff
10~11_takeoff
11~12_takeoff
18~20_ride
route_station
bus_route_id_weekday
station_code_weekday
route_station_weekday
68a
810a
1012a
68b
810b
1012b
1820_rs_mean
1820_rs_sum
1820_r_mean
1820_r_sum
1820_s_mean
1820_s_sum
1820_w_mean
1820_w_sum
mean_bus_weekday_ride
mean_station_weekday_ride
mean_route_station_weekday_ride
congestion
location
cue
1012a_sum
1012a_mean
1012b_sum
1012b_mean
10~11_ride_sum
10~11_ride_mean
10~11_takeoff_sum
10~11_takeoff_mean
11~12_ride_sum
11~12_ride_mean
11~12_takeoff_sum
11~12_takeoff_mean
1820_r_mean_sum
1820_r_mean_mean
1820_r_sum_sum
1820_r_sum_mean
1820_rs_mean_sum
1820_rs_mean_mean
1820_rs_sum_sum
1820_rs_sum_mean
1820_s_mean_sum
1820_s_mean_mean
1820_s_sum_sum
1820_s_sum_mean
1820_w_mean_sum
1820_w_mean_mean
1820_w_sum_sum
1820_w_sum_mean
68a_sum
68a_mean
68b_sum
68b_mean
6~

In [98]:
# 각 동, 읍, 면별 직업군별 비율, 평균소득액, 평균소비액 
# eju_finantial_life_data (외부데이터) 사용하여 피쳐 생성

# train, test 데이터의 'latitude', 'longitude'열, jeju_financial_life_data의 'X_AXIS', 'Y_AXIS'열을 지오코딩 프로그램을 사용하여 주소로 변환하였다. 변환한 주소를 전처리 하여 동 기준으로 두 데이터를 합쳐주었다. 
# 그 뒤 파생변수를 생성하였다.
# data_address.csv는 data의 주소가, life_address.csv는 jeju_finantial_life_data의 주소가 담겨있다.

def jeoju_love() :
    
    loc_data = pd.read_csv("data_address.csv", encoding='cp949')
    loc_life = pd.read_csv("life_address.csv", encoding='cp949')
    
    # location = coordi 
    loc_data = loc_data[['location','dong', 'si']]
    loc_life = loc_life[['location','dong', 'si']]

    df = pd.merge(data, loc_data, how='left', on='location')
    
    jeju_life = pd.read_csv("jeju_financial_life_data.csv")
    jeju_life['location'] = jeju_life['x_axis'].astype(str).str[:10] + ',' + jeju_life['y_axis'].astype(str).str[:10]
    jeju_life2 = pd.merge(jeju_life, loc_life, on='location')

    dong_f1 = jeju_life2.groupby(['dong'])[['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self','vehicle_own_rat', 'avg_income', 'med_income', 'avg_spend']].mean().reset_index()
    dong_f1.columns=['dong','mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 'mean_job_profession', 'mean_job_self','mean_vehicle_own_rat', 'mean_avg_income', 'mean_med_income', 'mean_avg_spend']

    dong_f2 = jeju_life2.groupby(['dong'])[['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self','vehicle_own_rat', 'avg_income', 'med_income', 'avg_spend']].sum().reset_index()
    dong_f2.columns=['dong','sum_job_majorc', 'sum_job_smallc', 'sum_job_public', 'sum_job_profession', 'sum_job_self','sum_vehicle_own_rat', 'sum_avg_income', 'sum_med_income', 'sum_avg_spend']

    dong_f3 = (jeju_life2.groupby(['dong'])['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self','vehicle_own_rat', 'avg_income', 'med_income', 'avg_spend'].sum()/jeju_life2.groupby(['dong'])['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self','vehicle_own_rat', 'avg_income', 'med_income', 'avg_spend'].sum().sum()).reset_index()
    dong_f3.columns = ['dong','rate_job_majorc', 'rate_job_smallc', 'rate_job_public', 'rate_job_profession', 'rate_job_self','rate_vehicle_own_rat', 'rate_avg_income', 'rate_med_income', 'rate_avg_spend']

    
    m_1 = pd.merge(dong_f1, dong_f2, how='left', on='dong')
    m_2 = pd.merge(m_1, dong_f3, how='left', on='dong')
    df = pd.merge(df, m_2, how='left', on='dong')

    #  # na preprocessing -> mean value
    df['mean_job_majorc'] = df['mean_job_majorc'].fillna(0.024219)
    df['mean_job_smallc'] = df['mean_job_smallc'].fillna(0.145757)
    df['mean_job_public'] = df['mean_job_public'].fillna(0.032768)
    df['mean_job_profession'] = df['mean_job_profession'].fillna(0.014855)
    df['mean_job_self'] = df['mean_job_self'].fillna(0.222090)
    df['mean_vehicle_own_rat'] = df['mean_vehicle_own_rat'].fillna(0.041161)
    df['mean_avg_income'] = df['mean_avg_income'].fillna(34221420)
    df['mean_med_income'] = df['mean_med_income'].fillna(30645290)
    df['mean_avg_spend'] = df['mean_avg_spend'].fillna(4224923)

    df['sum_job_majorc'] = df['sum_job_majorc'].fillna(3.717861e+00)
    df['sum_job_smallc'] = df['sum_job_smallc'].fillna(2.078142e+01)
    df['sum_job_public'] = df['sum_job_public'].fillna(4.747755e+00)
    df['sum_job_profession'] = df['sum_job_profession'].fillna(2.169554e+00)
    df['sum_job_self'] = df['sum_job_self'].fillna(3.044199e+01)
    df['sum_vehicle_own_rat'] = df['sum_vehicle_own_rat'].fillna(5.609080e+00)
    df['sum_avg_income'] = df['sum_avg_income'].fillna(4.998226e+09)
    df['sum_med_income'] = df['sum_med_income'].fillna(4.455924e+09)
    df['sum_avg_spend'] = df['sum_avg_spend'].fillna(6.147678e+08)

    df['rate_job_majorc'] = df['rate_job_majorc'].fillna(1.388889e-02)
    df['rate_job_smallc'] = df['rate_job_smallc'].fillna(1.388889e-02)
    df['rate_job_public'] = df['rate_job_public'].fillna(1.388889e-02)
    df['rate_job_profession'] = df['rate_job_profession'].fillna(1.388889e-02)
    df['rate_job_self'] = df['rate_job_self'].fillna(1.388889e-02)
    df['rate_vehicle_own_rat'] = df['rate_vehicle_own_rat'].fillna(1.388889e-02)
    df['rate_avg_income'] = df['rate_avg_income'].fillna(1.388889e-02)
    df['rate_med_income'] = df['rate_med_income'].fillna(1.388889e-02)
    df['rate_avg_spend'] = df['rate_avg_spend'].fillna(1.388889e-02)

    return df

In [99]:
data = jeoju_love()

data.shape

(643593, 172)

In [101]:
# 수요가 많을 것으로 예상되는 정류장
g = data[data['station_name'].str.contains('고등학교')]
highschool = list(g['station_name'].unique())

g = data[data['station_name'].str.contains('대학교')]
university = list(g['station_name'].unique())

In [102]:
def f(x):
    if x in highschool:
        return 1
    elif x in university:
        return 1
    else:
        return 0

In [103]:
data['school'] = data['station_name'].apply(f)

In [104]:
g = data[data['station_name'].str.contains('환승')]
transfer = list(g['station_name'].unique())

g = data[data['station_name'].str.contains('공항')]
airport = list(g['station_name'].unique())

g = data[data['station_name'].str.contains('터미널')]
terminal = list(g['station_name'].unique())

In [105]:
def f(x):
    if x in transfer:
        return 1
    elif x in airport:
        return 1
    elif x in terminal:
        return 1
    else:
        return 0

In [106]:
data['transfer'] = data['station_name'].apply(f) 

data.shape

(643593, 174)

In [107]:
# 동- 라벨 인코딩 
labelencoder = LabelEncoder()
df_encode = data[['dong']]
df_encoded = df_encode.apply(labelencoder.fit_transform)

data['dong2']=df_encoded['dong']

data.shape

(643593, 175)

In [108]:
# 측정소와 정류장 사이 거리 계산 
def dist() :
    jeju=(33.51411, 126.52969) # 제주 측정소 근처
    gosan=(33.29382, 126.16283) #고산 측정소 근처
    seongsan=(33.38677, 126.8802) #성산 측정소 근처
    po=(33.24616, 126.5653) #서귀포 측정소 근처

    t1 = [geopy.distance.vincenty( (i,j), jeju).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t2 = [geopy.distance.vincenty( (i,j), gosan).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t3 = [geopy.distance.vincenty( (i,j), seongsan).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t4 = [geopy.distance.vincenty( (i,j), po).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]

    data['dis_jeju'] = t1
    data['dis_gosan']=t2
    data['dis_seongsan']=t3
    data['dis_po']=t4

    total = pd.DataFrame(list(zip( t1,t2,t3,t4)),columns=['jeju','gosan','seongsan','po'] )
    data['dist_name'] = total.apply(lambda x: x.argmin(), axis=1)
    
    return data

In [109]:
data = dist()

data.shape 

(643593, 180)

In [133]:
data.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,route_station,bus_route_id_weekday,station_code_weekday,route_station_weekday,68a,810a,1012a,68b,810b,1012b,1820_rs_mean,1820_rs_sum,1820_r_mean,1820_r_sum,1820_s_mean,1820_s_sum,1820_w_mean,1820_w_sum,mean_bus_weekday_ride,mean_station_weekday_ride,mean_route_station_weekday_ride,congestion,location,cue,1012a_sum,1012a_mean,1012b_sum,1012b_mean,10~11_ride_sum,10~11_ride_mean,10~11_takeoff_sum,10~11_takeoff_mean,11~12_ride_sum,11~12_ride_mean,11~12_takeoff_sum,11~12_takeoff_mean,1820_r_mean_sum,1820_r_mean_mean,1820_r_sum_sum,1820_r_sum_mean,1820_rs_mean_sum,1820_rs_mean_mean,1820_rs_sum_sum,1820_rs_sum_mean,...,dis_jejuairport,dis_hallahosp,dis_rotary,dis_jejucenterhigh,dis_jejumarket,dis_jejusclass,dis_centerroad,dis_fiveway,dis_law,ride_sum,takeoff_sum,6~7_all_ride_number,7~8_all_ride_number,8~9_all_ride_number,9~10_all_ride_number,10~11_all_ride_number,weekend,holiday,weekdaymean,in_out_mean,adult,kids,teen,elder,adult_prop,kids_prop,teen_prop,elder_prop,dong,si,mean_job_majorc,mean_job_smallc,mean_job_public,mean_job_profession,mean_job_self,mean_vehicle_own_rat,mean_avg_income,mean_med_income,mean_avg_spend,sum_job_majorc,sum_job_smallc,sum_job_public,sum_job_profession,sum_job_self,sum_vehicle_own_rat,sum_avg_income,sum_med_income,sum_avg_spend,rate_job_majorc,rate_job_smallc,rate_job_public,rate_job_profession,rate_job_self,rate_vehicle_own_rat,rate_avg_income,rate_med_income,rate_avg_spend,school,transfer,dong2,dis_jeju,dis_gosan,dis_seongsan,dis_po,dist_name
0,0,2019-09-01,4270000,1,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"4270000,344","4270000,6","344,6","4270000,344,6",1.0,7.0,8.0,0.0,0.0,0.0,1.466667,44.0,3.104381,2409.0,1.466667,44.0,1.034282,54306.0,3.132812,0.200000,0.2,5,"33.4899,126.49373",0,244.0,5.304348,12.0,0.260870,136.0,2.956522,5.0,0.108696,108.0,2.347826,7.0,0.152174,142.801546,3.104381,110814.0,2409.0,67.466667,1.466667,2024.0,44.0,...,1.853545,0.719000,0.330360,4.226827,4.030090,1.897319,3.632769,1.243233,3.835351,16.0,0.0,2942.0,4518.0,5682.0,6275.0,6774.0,1,0,1.034282,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,NaN,NaN,0.024219,0.145757,0.032768,0.014855,0.22209,0.041161,34221420.0,30645290.0,4224923.0,3.717861,20.78142,4.747755,2.169554,30.44199,5.60908,4.998226e+09,4.455924e+09,614767800.0,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0,0,0,4.286659,37.692766,37.713630,27.841879,0
1,1,2019-09-01,4270000,1,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,"4270000,357","4270000,6","357,6","4270000,357,6",5.0,6.0,11.0,0.0,0.0,0.0,5.366667,161.0,3.104381,2409.0,4.178218,6330.0,1.034282,54306.0,3.132812,3.154867,2.2,5,"33.48944,126.48508",0,534.0,11.608696,17.0,0.369565,280.0,6.086957,6.0,0.130435,254.0,5.521739,11.0,0.239130,142.801546,3.104381,110814.0,2409.0,246.866667,5.366667,7406.0,161.0,...,2.057088,0.088058,1.109489,5.029806,4.704476,1.501101,4.311404,0.493820,4.640172,22.0,0.0,2942.0,4518.0,5682.0,6275.0,6774.0,1,0,1.034282,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,NaN,NaN,0.024219,0.145757,0.032768,0.014855,0.22209,0.041161,34221420.0,30645290.0,4224923.0,3.717861,20.78142,4.747755,2.169554,30.44199,5.60908,4.998226e+09,4.455924e+09,614767800.0,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0,0,0,4.966848,37.008492,38.465822,27.996179,0
2,2,2019-09-01,4270000,1,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,"4270000,432","4270000,6","432,6","4270000,432,6",2.0,2.0,0.0,0.0,0.0,0.0,1.600000,48.0,3.104381,2409.0,2.169559,1625.0,1.034282,54306.0,3.132812,1.163462,0.8,5,"33.48181,126.47352",0,106.0,2.304348,7.0,0.152174,43.0,0.934783,2.0,0.043478,63.0,1.369565,5.0,0.108696,142.80154

In [131]:
# 날씨 관련 변수 

rain3 = pd.read_csv("rain3.csv")

# train, test의 변수명과 통일시키고, NaN의 값은 0.0000으로 변경
rain3 = rain3.rename(columns={"일시":"date","지점":"dist_name"})
rain3 = rain3.fillna(0.00000)
rain3['date'] = pd.to_datetime(rain3['date'])

rain3.shape

(184, 4)

In [135]:
rain3['dist_name'].value_counts()
rain3['dist_name'] = rain3['dist_name'].map({'jeju':0,'gosan':1, 'seongsan': 2, 'po': 3})

rain3.head()

,dist_name,date,기온(°C),강수량(mm)
0,1,2019-09-01,22.018182,0.800000
1,1,2019-09-02,26.145455,0.700000
2,1,2019-09-03,25.254545,3.866667
3,1,2019-09-04,24.463636,0.266667
4,1,2019-09-05,27.136364,0.000000


In [136]:
data = pd.merge(data, rain3, how='left',on=['dist_name','date'])
data = pd.get_dummies(data,columns=['dist_name'])
data = pd.get_dummies(data,columns=['si'])

data.shape

(643593, 184)

In [0]:
# rainy_day
# 비 오는날=1, 비 안오는 날=0

In [137]:
def f(x):
    if x == 0:
        return 0
    else:
        return 1

In [138]:
data['rainy_day'] = data['강수량(mm)'].apply(f)

data.shape

(643593, 185)

In [139]:
# 승 하차 시간대 통합 변수 (t ~ t+3)
# t~t+1, t+1~t+2, t+2~t+3 시간대 승하차인원을 합하여 t~t+3 시간대 승하차인원 변수를 만듬

data['69a'] = data['6~7_ride']+data['7~8_ride']+data['8~9_ride']
data['912a']=data['9~10_ride']+data['10~11_ride']+data['11~12_ride']

data['69b'] = data['6~7_takeoff']+data['7~8_takeoff']+data['8~9_takeoff']
data['912b'] = data['9~10_takeoff']+data['10~11_takeoff']+data['11~12_takeoff']

data.shape

(643593, 189)

In [167]:
df_airport = pd.read_csv('df_airport.csv')

In [168]:
df_airport.head()

,date,domestic_tourists,international_tourists
0,2019-08-29,67712,6552
1,2019-08-30,73118,6904
2,2019-08-31,64752,6796
3,2019-09-01,64752,6524
4,2019-09-02,64752,6604


In [169]:
print(type(data['date'][3]))
print(type(df_airport['date'][3]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>


In [170]:
df_airport['date'] = pd.to_datetime(df_airport['date'])
print(type(df_airport['date'][3]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [171]:
print(type(data['date'][3]))
print(type(df_airport['date'][3]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [173]:
for col in df_airport.columns:
    print(col)

date
domestic_tourists
international_tourists


In [174]:
for col in data.columns: 
    print(col)

id
date
bus_route_id
in_out
station_code
station_name
latitude
longitude
6~7_ride
7~8_ride
8~9_ride
9~10_ride
10~11_ride
11~12_ride
6~7_takeoff
7~8_takeoff
8~9_takeoff
9~10_takeoff
10~11_takeoff
11~12_takeoff
18~20_ride
route_station
bus_route_id_weekday
station_code_weekday
route_station_weekday
68a
810a
1012a
68b
810b
1012b
1820_rs_mean
1820_rs_sum
1820_r_mean
1820_r_sum
1820_s_mean
1820_s_sum
1820_w_mean
1820_w_sum
mean_bus_weekday_ride
mean_station_weekday_ride
mean_route_station_weekday_ride
congestion
location
cue
1012a_sum
1012a_mean
1012b_sum
1012b_mean
10~11_ride_sum
10~11_ride_mean
10~11_takeoff_sum
10~11_takeoff_mean
11~12_ride_sum
11~12_ride_mean
11~12_takeoff_sum
11~12_takeoff_mean
1820_r_mean_sum
1820_r_mean_mean
1820_r_sum_sum
1820_r_sum_mean
1820_rs_mean_sum
1820_rs_mean_mean
1820_rs_sum_sum
1820_rs_sum_mean
1820_s_mean_sum
1820_s_mean_mean
1820_s_sum_sum
1820_s_sum_mean
1820_w_mean_sum
1820_w_mean_mean
1820_w_sum_sum
1820_w_sum_mean
68a_sum
68a_mean
68b_sum
68b_mean
6~

In [180]:
# df_airport와 merge 
data2 = pd.merge(pd.DataFrame(data), pd.DataFrame(df_airport), how = 'left', left_on = 'date', right_on = 'date')

# https://stackoverflow.com/questions/34227038/python-pandas-merge-keyerror
# 계속 KeyError 'date'
# drop_duplicates()는 너무 어려어서 관뒀고, 이게 강제로 할 수 있는 방법 중 최선이었음 

In [181]:
for col in data2.columns:
    print(col)

id
date
bus_route_id
in_out
station_code
station_name
latitude
longitude
6~7_ride
7~8_ride
8~9_ride
9~10_ride
10~11_ride
11~12_ride
6~7_takeoff
7~8_takeoff
8~9_takeoff
9~10_takeoff
10~11_takeoff
11~12_takeoff
18~20_ride
route_station
bus_route_id_weekday
station_code_weekday
route_station_weekday
68a
810a
1012a
68b
810b
1012b
1820_rs_mean
1820_rs_sum
1820_r_mean
1820_r_sum
1820_s_mean
1820_s_sum
1820_w_mean
1820_w_sum
mean_bus_weekday_ride
mean_station_weekday_ride
mean_route_station_weekday_ride
congestion
location
cue
1012a_sum
1012a_mean
1012b_sum
1012b_mean
10~11_ride_sum
10~11_ride_mean
10~11_takeoff_sum
10~11_takeoff_mean
11~12_ride_sum
11~12_ride_mean
11~12_takeoff_sum
11~12_takeoff_mean
1820_r_mean_sum
1820_r_mean_mean
1820_r_sum_sum
1820_r_sum_mean
1820_rs_mean_sum
1820_rs_mean_mean
1820_rs_sum_sum
1820_rs_sum_mean
1820_s_mean_sum
1820_s_mean_mean
1820_s_sum_sum
1820_s_sum_mean
1820_w_mean_sum
1820_w_mean_mean
1820_w_sum_sum
1820_w_sum_mean
68a_sum
68a_mean
68b_sum
68b_mean
6~

In [182]:
data2.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,route_station,bus_route_id_weekday,station_code_weekday,route_station_weekday,68a,810a,1012a,68b,810b,1012b,1820_rs_mean,1820_rs_sum,1820_r_mean,1820_r_sum,1820_s_mean,1820_s_sum,1820_w_mean,1820_w_sum,mean_bus_weekday_ride,mean_station_weekday_ride,mean_route_station_weekday_ride,congestion,location,cue,1012a_sum,1012a_mean,1012b_sum,1012b_mean,10~11_ride_sum,10~11_ride_mean,10~11_takeoff_sum,10~11_takeoff_mean,11~12_ride_sum,11~12_ride_mean,11~12_takeoff_sum,11~12_takeoff_mean,1820_r_mean_sum,1820_r_mean_mean,1820_r_sum_sum,1820_r_sum_mean,1820_rs_mean_sum,1820_rs_mean_mean,1820_rs_sum_sum,1820_rs_sum_mean,...,6~7_all_ride_number,7~8_all_ride_number,8~9_all_ride_number,9~10_all_ride_number,10~11_all_ride_number,weekend,holiday,weekdaymean,in_out_mean,adult,kids,teen,elder,adult_prop,kids_prop,teen_prop,elder_prop,dong,mean_job_majorc,mean_job_smallc,mean_job_public,mean_job_profession,mean_job_self,mean_vehicle_own_rat,mean_avg_income,mean_med_income,mean_avg_spend,sum_job_majorc,sum_job_smallc,sum_job_public,sum_job_profession,sum_job_self,sum_vehicle_own_rat,sum_avg_income,sum_med_income,sum_avg_spend,rate_job_majorc,rate_job_smallc,rate_job_public,rate_job_profession,rate_job_self,rate_vehicle_own_rat,rate_avg_income,rate_med_income,rate_avg_spend,school,transfer,dong2,dis_jeju,dis_gosan,dis_seongsan,dis_po,기온(°C),강수량(mm),dist_name_0,dist_name_1,dist_name_2,dist_name_3,rainy_day,69a,912a,69b,912b,domestic_tourists,international_tourists
0,0,2019-09-01,4270000,1,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"4270000,344","4270000,6","344,6","4270000,344,6",1.0,7.0,8.0,0.0,0.0,0.0,1.466667,44.0,3.104381,2409.0,1.466667,44.0,1.034282,54306.0,3.132812,0.200000,0.2,5,"33.4899,126.49373",0,244.0,5.304348,12.0,0.260870,136.0,2.956522,5.0,0.108696,108.0,2.347826,7.0,0.152174,142.801546,3.104381,110814.0,2409.0,67.466667,1.466667,2024.0,44.0,...,2942.0,4518.0,5682.0,6275.0,6774.0,1,0,1.034282,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,NaN,0.024219,0.145757,0.032768,0.014855,0.22209,0.041161,34221420.0,30645290.0,4224923.0,3.717861,20.78142,4.747755,2.169554,30.44199,5.60908,4.998226e+09,4.455924e+09,614767800.0,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0,0,0,4.286659,37.692766,37.713630,27.841879,22.727273,2.272727,1,0,0,0,1,3.0,13.0,0.0,0.0,64752,6524
1,1,2019-09-01,4270000,1,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,"4270000,357","4270000,6","357,6","4270000,357,6",5.0,6.0,11.0,0.0,0.0,0.0,5.366667,161.0,3.104381,2409.0,4.178218,6330.0,1.034282,54306.0,3.132812,3.154867,2.2,5,"33.48944,126.48508",0,534.0,11.608696,17.0,0.369565,280.0,6.086957,6.0,0.130435,254.0,5.521739,11.0,0.239130,142.801546,3.104381,110814.0,2409.0,246.866667,5.366667,7406.0,161.0,...,2942.0,4518.0,5682.0,6275.0,6774.0,1,0,1.034282,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,NaN,0.024219,0.145757,0.032768,0.014855,0.22209,0.041161,34221420.0,30645290.0,4224923.0,3.717861,20.78142,4.747755,2.169554,30.44199,5.60908,4.998226e+09,4.455924e+09,614767800.0,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0,0,0,4.966848,37.008492,38.465822,27.996179,22.727273,2.272727,1,0,0,0,1,9.0,13.0,0.0,0.0,64752,6524
2,2,2019-09-01,4270000,1,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,"4270000,432","4270000,6","432,6","4270000,432,6",2.0,2.0,0.0,0.0,0.0,0.0,1.600000,48.0,3.104381,2409.0,2.169559,1625.0,1.034282,54306.0,3.132812,1.163462,0.8,5,"33.48181,126.47352",0,106.0,2.304348,7.0,0.152174,43.0,0.934783,2.0,0.043478,63.0,1.369565,5.0,0.108696,142.801546,3.104381,110814.0,2409.0,73.600000,1.600000,2208.0,48.0,...,2942.0,4518.0,5682.0,6275.0,6774.0,1,0,1.03

In [184]:
# Make dataset
# train, test 데이터를 만들어준다
train_data2 = data2.query('cue=="0"').reset_index()
test_data2 = data2.query('cue=="1"').reset_index()

train_data2.shape, test_data2.shape
# 요약통계량을 통한 변수를 만드는 과정에서 NA가 발생한다.

# target variable을 분리한 후 데이터를 저장한다.

((415423, 192), (228170, 192))

In [185]:

y_train2 = train_data2[['18~20_ride']]

In [186]:
train_data2.shape, test_data2.shape, y_train2.shape

((415423, 192), (228170, 192), (415423, 1))

In [187]:

train_data2.to_csv('X_train2.csv', index = False, encoding = 'utf-8')
test_data2.to_csv('X_test2.csv', index = False, encoding = 'utf-8')
y_train2.to_csv('y_train2.csv', index = False, encoding = 'utf-8')

print('save ..!')

save ..!


In [188]:
train_data2 = pd.read_csv('X_train2.csv', encoding = 'utf-8')
test_data2 = pd.read_csv('X_test2.csv', encoding = 'utf-8')
y_train2 = pd.read_csv('y_train2.csv', encoding = 'utf-8')

Columns 

In [191]:
input_var_0 = []

for col in train_data2.columns: 
      input_var_0.append(col)

In [192]:
print(input_var_0)

['index', 'id', 'date', 'bus_route_id', 'in_out', 'station_code', 'station_name', 'latitude', 'longitude', '6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff', '18~20_ride', 'route_station', 'bus_route_id_weekday', 'station_code_weekday', 'route_station_weekday', '68a', '810a', '1012a', '68b', '810b', '1012b', '1820_rs_mean', '1820_rs_sum', '1820_r_mean', '1820_r_sum', '1820_s_mean', '1820_s_sum', '1820_w_mean', '1820_w_sum', 'mean_bus_weekday_ride', 'mean_station_weekday_ride', 'mean_route_station_weekday_ride', 'congestion', 'location', 'cue', '1012a_sum', '1012a_mean', '1012b_sum', '1012b_mean', '10~11_ride_sum', '10~11_ride_mean', '10~11_takeoff_sum', '10~11_takeoff_mean', '11~12_ride_sum', '11~12_ride_mean', '11~12_takeoff_sum', '11~12_takeoff_mean', '1820_r_mean_sum', '1820_r_mean_mean', '1820_r_sum_sum', '1820_r_sum_mean', '1820_rs_mean_sum', '1820_rs_mean_mean'

# Ensemble of Ensemble Learner 만들기 (산술평균) 


In [201]:
bagging_predict_result = []

for _ in range(10):
    data_index = [data_index for data_index in range(X.shape[0])]
    random_data_index = np.random.choice(data_index, X.shape[0])
    print(len(set(random_data_index)))
    lgb_dtrain = lgb.Dataset(data = X.iloc[random_data_index.l, label = y.iloc[random_data_index]])
    lgb_param = {'max_depth':14, 
                 'learning_rate': 0.01, 
                 'n_estimator': 500, 
                  'objective':'regression',
                  'n_jjobs': '-1',}
    lgb_model = lgb.train(prams = lgb_param, train_set = lgb.dtrain)
    predict1 = lgb_model.predict(X_test[input_var_0], axis = 1)
    bagging_predict_result.append(predict)

SyntaxError: invalid syntax (<ipython-input-201-31cd7f0696ba>, line 7)

# 모델별 결과값 평균내기 (산술평균) 

In [0]:
bagging_predict = []
for list2_index in range(X_test[input_var_0], shape[0]): 
    temp_predict = []
    for list_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[list_index][list2_index])
    bagging_predict.append[X_test.iloc[list2_index['id'].np.mean(temp_predict)]]

bagging_predict 

# Ensemble of Ensemble Learner 만들기 (가중평균) 

In [0]:
bagging_predict_result = [ ]
bagging_predict_result0 = []

for _ in range(10): 
    data_index = [data_index for data_index in range(X.shape[0])]
    random_data_index = np.random.choice(data_index, X.shape[0])
    print(len(set(random_data_index)))
    lgb_dtrain = lgb.Dataset(data = X.iloc[random_data_index].label - y.iloc[random_data_index,])
    lgb_param = {'max_depth': 14,
               'learning_rate': 0.01, 
               'n_estimator': 500, 
               'objective': 'regression',
               'n_jobs': '-1'}
    lgb_model = lgb.train(param = lgb_param, train_set = lgb_dtrain)

    predict0 = lgb_model.predict(X_axis = 1)
    predict1 = lgb_model.predict(X_test[input_var_0], axis=1)

    bagging_predict_result0.accend(predict0)
    bagging_predict_result.append(predict1)

# 모델별 결과값 평균내기 (가중평균) 

In [200]:
bagging_predict = []
bagging_rmse = []

for list_index in range(bagging_predict_result0):
    bagging_rmse.append(sort(mean_squared_error(bagging_predict_result0[list_index], y)))

temp_sum = bagging_rmse/np.sum(bagging_rmse)
temp_sum_sum = np.sum(temp_sum)
temp_ratio = temp_sum/temp_sum_sum

for list2_index in range(X_test.shape[0]): 
    temp_predict = []
    for list_index in range(len(bagging_predict_result)):
        temp_predict.accending(bagging_predict_result[list.index][list2.index])
    bagging_predict.append(X_test.iloc[list2_index]['id'].np.dot(temp_ratio, np.transpose(temp_predict))) 

NameError: name 'bagging_predict_result0' is not defined

# HYPERPARAMETER + gird & random search 적용 후 

In [0]:
# Manual: grid search and radom search 
# codes from https://www.kaggle.com/mlisovyi/lightgbm-hyperparameter-optimisation-lb-0-761 

# split the full sample into train/test 
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.20, random_state=314, stratify=y_rus)

In [ ]:
input_var=['in_out','latitude', 'longitude', '68a', '810a', '1012a', '68b', '810b', '1012b',
           'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
           'dis_jeju', 'dis_gosan','dis_seongsan', 'dis_po','기온(°C)', '강수량(mm)', 
           'dist_name_gosan', 'dist_name_jeju','dist_name_po', 'dist_name_seongsan']

target=['18~20_ride']

In [ ]:
X_train=train2[input_var]
random.seed(1217) #동일한 샘플링하기 위한 시드번호
train_list=random.sample(list(range(X_train.shape[0])), int(round(X_train.shape[0]*0.01,0)) )

X_train=train2[input_var]
X_train=X_train.iloc[train_list,:]
y_train=train2[target]
y_train=y_train.iloc[train_list,:]

X_test=test2[input_var]

X_train.shape, y_train.shape

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_features': [2,3,5],
    'min_samples_leaf': [2,3],
    'min_samples_split': [2,4,6],
    'n_estimators': [100, 200,500]
}
# Create a based model
rf = RandomForestRegressor(random_state=1217) # 랜덤포레스트 모델을 정의한다.
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid) # GridSearchCV를 정의한다.

grid_search.fit(X_train, y_train)

grid_search.best_params_ #학습 이후 최적의 paramter를 출력

#해당 코드 실행시간 2분 ~ 3분 소요

In [ ]:
#전체 데이터로 적용
X_train=train2[input_var]
y_train=train2[target]

X_test=test2[input_var]

X_train.shape, y_train.shape, X_test.shape

In [ ]:
# best_params_를 통해서 나온 값을 투입
rf = RandomForestRegressor(max_features=3,min_samples_leaf=2,min_samples_split=2,n_estimators=500,random_state=1217)

rf.fit(X_train,y_train) #학습 

test['18~20_ride'] = rf.predict(X_test) #예측값 생성 후, test['18~20_ride']에 집어 넣는다.

test[['id','18~20_ride']].to_csv("dacon_base_middle2.csv",index=False) # id와 18~20_ride만 선택 후 csv 파일로 내보낸다

#해당 코드 소요 시간 5분

# Bayesian Optimization 활용 
https://dacon.io/competitions/official/235583/codeshare/703

In [0]:
# # Bayesian 
# def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
#     score = 0
#     kf = KFold(n_splits=n_splits)
#     models = []
#     for train_index, valid_index in kf.split(x_data):
#         x_train, y_train = x_data.iloc[train_index], y_data[train_index]
#         x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
#         model = lgb.LGBMClassifier(
#             num_leaves = int(num_leaves), 
#             learning_rate = learning_rate, 
#             n_estimators = int(n_estimators), 
#             subsample = np.clip(subsample, 0, 1), 
#             colsample_bytree = np.clip(colsample_bytree, 0, 1), 
#             reg_alpha = reg_alpha, 
#             reg_lambda = reg_lambda,
#         )
        
#         model.fit(x_train, y_train)
#         models.append(model)
        
#         pred = model.predict_proba(x_valid)[:, 1]
#         true = y_valid
#         score += roc_auc_score(true, pred)/n_splits
    
#     if output == 'score':
#         return score
#     if output == 'model':
#         return models

In [0]:
# # 모델과 관련없는 변수 고정
# func_fixed = partial(lgb_cv, x_data=x_train, y_data=y_train, n_splits=5, output='score') 
# # 베이지안 최적화 범위 설정
# lgbBO = BayesianOptimization(
#     func_fixed, 
#     {
#         'num_leaves': (16, 1024),        # num_leaves,       범위(16~1024)
#         'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
#         'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
#         'subsample': (0, 1),             # subsample,        범위(0~1)
#         'colsample_bytree': (0, 1),      # colsample_bytree, 범위(0~1)
#         'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
#         'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
#     }, 
#     random_state=4321                    # 시드 고정
# )
# lgbBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
# LightGBM Classifier: https://lightgbm.readthedocs.io/en/latest/pythonapi/